In [ ]:
print("----\n")

In [9]:
##%overwritefile
##%file:src/compile_with_kotlinc.py
##%noruncode
    def compile_with_kotlinc(self, source_filename, binary_filepath=None, cflags=None, ldflags=None,env=None,coptions=None):
        # coptions = ['-b', '', '-cp', '', '-j'] + cflags
        outpath=os.path.dirname(source_filename)
        sf = os.path.basename(source_filename)
        binary_filename = sf.split(".")[0]
        # if self.linkMaths:
        #     cflags = cflags + ['-lm']
        # if self.wError:
        #     cflags = cflags + ['-Werror']
        # if self.wAll:
        #     cflags = cflags + ['-Wall']
        # if self.readOnlyFileSystem:
        #     cflags = ['-DREAD_ONLY_FILE_SYSTEM'] + cflags
        # if self.bufferedOutput:
        #     cflags = ['-DBUFFERED_OUTPUT'] + cflags
        index=-1
        if coptions==None:
            coptions=[]
        for s in coptions:
            index=index+1
            if s=='-d':
                outpath=coptions[index+1]
                if not outpath.startswith('-'):
                    #剔除 -d参数和值
                    outpath=coptions.pop(index+1)
                    coptions.pop(index)
            else:
                if binary_filepath!=None:
                    outpath=binary_filepath
        args = ['kotlinc']+coptions+ ['-d', outpath]+[ source_filename]
        self._log(' '.join((' '+ str(s) for s in args))+"\n")
        binary_filename=binary_filename.capitalize()
        binary_filename=os.path.join(outpath,binary_filename)
        binary_filename=binary_filename+"Kt" 
        # self._log("binary_filename:"+binary_filename+"\n")
        return self.create_jupyter_subprocess(args,env=env),binary_filename+".class",args


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/src/compile_with_kotlinc.py created successfully


In [6]:
##%overwritefile
##%file:src/k_exec_kotlinc_.py
##%noruncode
    def _exec_kotlinc_(self,source_filename,magics):
        self._write_to_stdout('Generating binary file\n')
        p,outfile,gcccmd = self.compile_with_kotlinc(
            source_filename, 
            None,
            self.addkey2dict(magics,'cflags'),
            self.addkey2dict(magics,'ldflags'),
            self.addkey2dict(magics,'env'),
            self.addkey2dict(magics,'coptions'),
            )
        while p.poll() is None:
            p.write_contents()
        p.write_contents()
        if p.returncode != 0:  # Compilation failed
            self._log(''.join((str(s) for s in ccmd))+"\n",3)
            self._log("Kotlin exited with code {}, the executable will not be executed".format(p.returncode),3)

            # delete source files before exit
            os.remove(source_filename)
            # os.remove(binary_file.name)
        return p.returncode,outfile


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/src/k_exec_kotlinc_.py created successfully
